In [1]:
import pandas as pd
import numpy as np
from haversine import haversine
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint
from sklearn.metrics import mean_squared_error, r2_score
import math
from numba import jit
import math
import winsound
def play_chime():
    # The frequency and duration of the chime (in Hertz and milliseconds)
    frequency = 1000  # Frequency of the chime in Hertz
    duration = 300  # Duration of the chime in milliseconds

    # Play the chime
    winsound.Beep(frequency, duration)

if __name__ == "__main__":
    play_chime()

In [2]:

@jit(nopython=True)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

def categorize_hour(time):
    hour = time.hour
    return f'{hour:02d}-{(hour + 1) % 24:02d}'

if __name__ == "__main__":
    play_chime()

In [3]:
train = pd.read_csv(r'C:\Users\archi\OneDrive\Desktop\Python Projects\Kaggle\NYC Taxi Trip Duration\train.csv', parse_dates = True)

In [4]:
df = train.copy()

In [5]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['pickup_date'] = df['pickup_datetime'].dt.date
df['pickup_time'] = df['pickup_datetime'].dt.strftime('%H:%M:%S')
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
df['dropoff_date'] = df['dropoff_datetime'].dt.date
df['dropoff_time'] = df['dropoff_datetime'].dt.strftime('%H:%M:%S')
df['trip_duration'] = df['trip_duration']/60
df['pickup_year'] = df['pickup_datetime'].dt.year
df['pickup_day'] = df['pickup_datetime'].dt.day
df['pickup_month'] = df['pickup_datetime'].dt.month
df['dropoff_year'] = df['dropoff_datetime'].dt.year
df['dropoff_day'] = df['dropoff_datetime'].dt.day
df['dropoff_month'] = df['dropoff_datetime'].dt.month
df['Distance'] = df.apply(
    lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']),
    axis=1
)
df['Speed'] = df['Distance']/df['trip_duration']
df['pickup_time'] = pd.to_datetime(df['pickup_time'], format='%H:%M:%S').dt.time
df['pickup_category_time'] = df['pickup_time'].apply(categorize_hour)

In [6]:
del train

In [7]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,dropoff_time,pickup_year,pickup_day,pickup_month,dropoff_year,dropoff_day,dropoff_month,Distance,Speed,pickup_category_time
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,17:32:30,2016,14,3,2016,14,3,1.498521,0.197607,17-18
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,00:54:38,2016,12,6,2016,12,6,1.805507,0.163394,00-01
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,12:10:48,2016,19,1,2016,19,1,6.385098,0.180370,11-12
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,19:39:40,2016,6,4,2016,6,4,1.485498,0.207762,19-20
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,13:38:10,2016,26,3,2016,26,3,1.188588,0.163943,13-14


In [8]:
#df[['pickup_datetime', 'dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'trip_duration']].head()

In [9]:
dum_1 = pd.get_dummies(df['pickup_category_time'],prefix = 'pickup_category_time_' )

df = pd.concat([df, dum_1], axis = 1)

In [10]:
df.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'pickup_date', 'pickup_time', 'dropoff_date',
       'dropoff_time', 'pickup_year', 'pickup_day', 'pickup_month',
       'dropoff_year', 'dropoff_day', 'dropoff_month', 'Distance', 'Speed',
       'pickup_category_time', 'pickup_category_time__00-01',
       'pickup_category_time__01-02', 'pickup_category_time__02-03',
       'pickup_category_time__03-04', 'pickup_category_time__04-05',
       'pickup_category_time__05-06', 'pickup_category_time__06-07',
       'pickup_category_time__07-08', 'pickup_category_time__08-09',
       'pickup_category_time__09-10', 'pickup_category_time__10-11',
       'pickup_category_time__11-12', 'pickup_category_time__12-13',
       'pickup_category_time__13-14', 'pickup_category_time__14-15',
       'pickup_category_time__15-16', 'pi

In [11]:
df_2 = df[df['trip_duration'] < 5000]
df_2 = df.drop(['id','vendor_id','pickup_year', 'trip_duration', 'pickup_time', 'pickup_date','dropoff_date',
       'dropoff_time', 'pickup_year', 'dropoff_year',
       'pickup_category_time', 'pickup_datetime', 'dropoff_datetime'], axis = 1)

In [12]:
df_2.columns

Index(['passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'pickup_day', 'pickup_month', 'dropoff_day', 'dropoff_month',
       'Distance', 'Speed', 'pickup_category_time__00-01',
       'pickup_category_time__01-02', 'pickup_category_time__02-03',
       'pickup_category_time__03-04', 'pickup_category_time__04-05',
       'pickup_category_time__05-06', 'pickup_category_time__06-07',
       'pickup_category_time__07-08', 'pickup_category_time__08-09',
       'pickup_category_time__09-10', 'pickup_category_time__10-11',
       'pickup_category_time__11-12', 'pickup_category_time__12-13',
       'pickup_category_time__13-14', 'pickup_category_time__14-15',
       'pickup_category_time__15-16', 'pickup_category_time__16-17',
       'pickup_category_time__17-18', 'pickup_category_time__18-19',
       'pickup_category_time__19-20', 'pickup_category_time__20-21',
       'pickup_category_time__21-22', 'pickup_c

In [13]:
dum_1 = pd.get_dummies(df_2['pickup_day'],prefix = 'pickup_day' )
dum_2 = pd.get_dummies(df_2['dropoff_day'],prefix = 'dropoff_day')
dum_3 = pd.get_dummies(df_2['pickup_month'],prefix = 'pickup_month' )
dum_4 = pd.get_dummies(df_2['dropoff_month'],prefix = 'dropoff_month')
dum_5 = pd.get_dummies(df_2['store_and_fwd_flag'], prefix = 'flag_')
df_2 = df_2.drop(['store_and_fwd_flag'], axis = 1)

df_2 = pd.concat([df_2, dum_1, dum_2, dum_3, dum_4, dum_5],axis = 1)

In [14]:
# from sklearn.cluster import KMeans

# kmeans_pickup = KMeans(n_clusters=18, random_state=42)
# df_2['pickup_cluster'] = kmeans_pickup.fit_predict(df_2[['pickup_latitude', 'pickup_longitude']])

# kmeans_dropoff = KMeans(n_clusters=18, random_state=42)
# df_2['dropoff_cluster'] = kmeans_dropoff.fit_predict(df_2[['dropoff_latitude', 'dropoff_longitude']])



In [15]:
# # Visualize the clusters
# plt.figure(figsize=(10, 6))

# # Plot pickup clusters
# plt.subplot(1, 2, 1)
# plt.scatter(df_2['pickup_longitude'], df_2['pickup_latitude'], c=df_2['pickup_cluster'], cmap='rainbow', marker='o')
# plt.title('Pickup Clusters')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')

# # Plot dropoff clusters
# plt.subplot(1, 2, 2)
# plt.scatter(df_2['dropoff_longitude'], df_2['dropoff_latitude'], c=df_2['dropoff_cluster'], cmap='rainbow', marker='o')
# plt.title('Dropoff Clusters')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')

# plt.tight_layout()
# plt.show()



In [16]:
# dum_1 = pd.get_dummies(df_2['pickup_cluster'],prefix = 'pickup_cluster' )
# dum_2 = pd.get_dummies(df_2['dropoff_cluster'],prefix = 'dropoff_cluster')
# df_2 = df_2.drop(['pickup_cluster', 'dropoff_cluster', 'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','pickup_day','pickup_month','dropoff_day','dropoff_month'], axis = 1)
# df_2 = pd.concat([df_2, dum_1, dum_2],axis = 1)

In [17]:

df_2 = df_2.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','pickup_day','pickup_month','dropoff_day','dropoff_month'], axis = 1)
df_2 = pd.concat([df_2, dum_1, dum_2],axis = 1)

In [18]:
list(df_2.columns)

['passenger_count',
 'Distance',
 'Speed',
 'pickup_category_time__00-01',
 'pickup_category_time__01-02',
 'pickup_category_time__02-03',
 'pickup_category_time__03-04',
 'pickup_category_time__04-05',
 'pickup_category_time__05-06',
 'pickup_category_time__06-07',
 'pickup_category_time__07-08',
 'pickup_category_time__08-09',
 'pickup_category_time__09-10',
 'pickup_category_time__10-11',
 'pickup_category_time__11-12',
 'pickup_category_time__12-13',
 'pickup_category_time__13-14',
 'pickup_category_time__14-15',
 'pickup_category_time__15-16',
 'pickup_category_time__16-17',
 'pickup_category_time__17-18',
 'pickup_category_time__18-19',
 'pickup_category_time__19-20',
 'pickup_category_time__20-21',
 'pickup_category_time__21-22',
 'pickup_category_time__22-23',
 'pickup_category_time__23-00',
 'pickup_day_1',
 'pickup_day_2',
 'pickup_day_3',
 'pickup_day_4',
 'pickup_day_5',
 'pickup_day_6',
 'pickup_day_7',
 'pickup_day_8',
 'pickup_day_9',
 'pickup_day_10',
 'pickup_day_11',


In [19]:
y = df_2[['Speed']].copy()
X = df_2.drop(['Speed'], axis = 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Random Forest

In [21]:
# param_grid = {
#     'n_estimators': randint(100, 1000),  # Number of trees in the forest
#     'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at every split
#     'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],  # Maximum number of levels in tree
#     'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
#     'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
#     'bootstrap': [True, False]  # Method of selecting samples for training each tree
# }


# # Perform Randomized Search Cross-Validation
# random_search = RandomizedSearchCV(estimator=rf_regressor, param_distributions=param_grid, 
#                                    n_iter=50, cv=KFold(n_splits=5, shuffle=True), 
#                                    scoring='neg_mean_squared_error', random_state=42, 
#                                    n_jobs=-4)

# # Fit the model
# random_search.fit(X, y)

# # Get the best parameters and best score
# best_params = random_search.best_params_
# best_score = random_search.best_score_

# # Predict on the test set
# y_pred = random_search.predict(X_test)

# # Calculate Mean Squared Error (MSE) and R-squared
# mse = mean_squared_error(y_test, y_pred)
# r_squared = r2_score(y_test, y_pred)

# print("Best Parameters:", best_params)
# print("Best Score (neg_mean_squared_error):", best_score)
# print("Mean Squared Error (MSE):", mse)
# print("R-squared:", r_squared)


### Decsision Tree

In [23]:
param_dist = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None] + list(np.arange(1, 21)),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2', None]
}

from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(random_state=42)

# Set up RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=clf,
#     param_distributions=param_dist,
#     n_iter=10,  # Number of parameter settings sampled
#     cv=5,  # Number of cross-validation folds
#     random_state=42,
#     n_jobs=-1  # Use all available CPU cores
# )

In [24]:
clf.fit(X_train, y_train)


def play_chime():
    # The frequency and duration of the chime (in Hertz and milliseconds)
    frequency = 1000  # Frequency of the chime in Hertz
    duration = 500  # Duration of the chime in milliseconds

    # Play the chime
    winsound.Beep(frequency, duration)

if __name__ == "__main__":
    play_chime()

In [25]:
# Get feature importances
feature_importances = clf.feature_importances_

# Print feature importances
for feature, importance in zip(feature_names, feature_importances):
    print(f"{feature}: {importance:.4f}")

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature, feature_importances, align='center')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importances in Decision Tree')
plt.show()

NameError: name 'feature_names' is not defined

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

# Assuming df_2 is already defined and contains the relevant data
# Extract the target variable and features
y = df_2[['Speed']].copy()
X = df_2.drop(['Speed'], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter distribution for RandomizedSearchCV (not used in this example)

# Initialize and fit the DecisionTreeRegressor
clf = DecisionTreeRegressor(random_state=42)
clf.fit(X_train, y_train)

# Get feature importances
feature_importances = clf.feature_importances_
feature_names = X.columns  # Extract feature names from the DataFrame

# Print feature importances
for feature, importance in zip(feature_names, feature_importances):
    print(f"{feature}: {importance:.4f}")

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_names, feature_importances, align='center')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importances in Decision Tree')
plt.show()


In [ ]:
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

# Print the DataFrame
print(feature_importances_df)

In [ ]:
feature_importances_df.head(10)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on the training set
y_train_pred = clf.predict(X_train)

# Predict on the test set
y_test_pred = clf.predict(X_test)

# Compute Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# Compute R² score
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)


In [ ]:
# Print metrics
print(f"Mean Squared Error (Train): {mse_train:.4f}")
print(f"Mean Squared Error (Test): {mse_test:.4f}")
print(f"R² Score (Train): {r2_train:.4f}")
print(f"R² Score (Test): {r2_test:.4f}")